In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from arch import arch_model
from scipy import stats
import pymc as pm
import arviz as az
import sys
import pytensor.tensor as pt
from pytensor.scan import scan

sys.path.append('../src')
from utils import (
    train_test_split_timeseries,
    extract_stock_returns,
    create_results_directory,
    print_convergence_diagnostics,
    plot_mcmc_diagnostics
)
from priors import AR1_priors, GARCH_priors,SV_priors

np.random.seed(20251127)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
az.style.use('arviz-darkgrid')

results_dirs = create_results_directory(base_dir='../results/03_basic_models')

Created results directory structure in ../results/03_basic_models


# 1 DATA PREPARATION


In [3]:
returns_df = pd.read_csv('../data/processed/log_returns.csv', index_col=0, parse_dates=True)
train_df, test_df, split_date = train_test_split_timeseries(returns_df, train_ratio=0.8)
stock_list = returns_df.columns.tolist()

print(f"Total stocks: {len(stock_list)}")
print(f"Train period: {train_df.index[0]} to {train_df.index[-1]}")
print(f"Test period: {test_df.index[0]} to {test_df.index[-1]}")

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
for idx, stock in enumerate(stock_list[:3]):
    ax = axes[idx]
    ax.plot(train_df.index, train_df[stock], 'b-', label='Train', alpha=0.7, linewidth=1)
    ax.plot(test_df.index, test_df[stock], 'r-', label='Test', alpha=0.7, linewidth=1)
    ax.axvline(split_date, color='black', linestyle='--', linewidth=2, alpha=0.5)
    ax.set_title(f'{stock}: Train-Test Split')
    ax.set_xlabel('Date')
    ax.set_ylabel('Log Return')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{results_dirs['figures']}/01_train_test_split.png", dpi=300, bbox_inches='tight')
plt.close()

print("Data preparation complete.")

Train-Test Split:
  Train: 2020-11-25 to 2024-11-20
  Test:  2024-11-21 to 2025-11-21
  Train size: 1003 observations
  Test size: 251 observations
Total stocks: 7
Train period: 2020-11-25 00:00:00 to 2024-11-20 00:00:00
Test period: 2024-11-21 00:00:00 to 2025-11-21 00:00:00
Data preparation complete.


# 2: AR(1) MODEL - FREQUENTIST ESTIMATION

In [7]:
print("\nFitting AR(1) using MLE...")

ar1_freq_results = {}
ar1_freq_params = {}

for stock in stock_list:
    returns_train = train_df[stock].values
    model = AutoReg(returns_train, lags=1, seasonal=False, trend='c')
    results = model.fit()
    
    ar1_freq_results[stock] = {
        'model': model,
        'results': results,
        'returns_train': returns_train,
        'returns_test': test_df[stock].values
    }
    
    params = results.params
    mu = params[0]
    phi = params[1]
    sigma_sq = results.resid.var()
    
    ar1_freq_params[stock] = {
        'mu': mu,
        'phi': phi,
        'sigma_sq': sigma_sq,
        'sigma': np.sqrt(sigma_sq),
        'aic': results.aic,
        'bic': results.bic,
        'log_likelihood': results.llf
    }

ar1_freq_summary = pd.DataFrame(ar1_freq_params).T
ar1_freq_summary = ar1_freq_summary[['mu', 'phi', 'sigma', 'log_likelihood', 'aic']]

print("\nAR(1) FREQUENTIST RESULTS:")
print(ar1_freq_summary.to_string())

ar1_freq_summary.to_csv(f"{results_dirs['ar1_freq']}/ar1_frequentist_summary.csv")

# Diagnostics
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for idx, stock in enumerate(stock_list[:2]):
    results = ar1_freq_results[stock]['results']
    resid = results.resid
    
    ax = axes[idx, 0]
    ax.plot(resid, 'b-', alpha=0.7, linewidth=1)
    ax.axhline(0, color='red', linestyle='--', alpha=0.5)
    ax.set_title(f'{stock}: Residuals')
    ax.set_ylabel('Residual')
    ax.grid(True, alpha=0.3)
    
    ax = axes[idx, 1]
    sm.graphics.tsa.plot_acf(resid, lags=20, ax=ax)
    ax.set_title(f'{stock}: ACF of Residuals')
    
    ax = axes[idx, 2]
    ax.hist(resid, bins=50, edgecolor='black', alpha=0.7)
    ax.set_title(f'{stock}: Residual Distribution')
    ax.set_xlabel('Residual')
    ax.set_ylabel('Frequency')
    
    ax = axes[idx, 3]
    stats.probplot(resid, dist="norm", plot=ax)
    ax.set_title(f'{stock}: Q-Q Plot')

plt.tight_layout()
plt.savefig(f"{results_dirs['ar1_freq']}/ar1_frequentist_diagnostics.png", dpi=300, bbox_inches='tight')
plt.close()


Fitting AR(1) using MLE...

AR(1) FREQUENTIST RESULTS:
             mu       phi     sigma  log_likelihood          aic
AAPL   0.000700  0.001282  0.016902     2666.716664 -5327.433329
AMZN   0.000242  0.000464  0.022156     2395.497315 -4784.994630
GOOGL  0.000696 -0.005942  0.019166     2540.770824 -5075.541648
META   0.000726 -0.008070  0.028492     2143.477688 -4280.955375
MSFT   0.000706 -0.016698  0.016395     2697.242830 -5388.485660
NVDA   0.002476 -0.032073  0.032823     2001.676721 -3997.353441
TSLA   0.000598 -0.029272  0.037587     1865.887725 -3725.775449


# 3: AR(1) MODEL - BAYESIAN ESTIMATION

In [8]:
def build_ar1_bayesian_model(returns):
    """
    AR(1) Bayesian Model
    Model: r_t = mu + phi * r_{t-1} + epsilon_t
    where epsilon_t ~ N(0, sigma^2)
    """
    prior_spec = AR1_priors.get_pymc_spec()
    
    with pm.Model() as model:
        # Priors
        mu = pm.Normal('mu', 
                      mu=prior_spec['mu'][1]['mu'], 
                      sigma=prior_spec['mu'][1]['sigma'])
        
        phi = pm.Normal('phi', 
                       mu=prior_spec['phi'][1]['mu'], 
                       sigma=prior_spec['phi'][1]['sigma'])
        
        sigma_sq = pm.HalfCauchy('sigma_sq', 
                                beta=prior_spec['sigma_sq'][1]['beta'])
        
        # Likelihood
        r_lag = returns[:-1]
        r_curr = returns[1:]
        mu_t = mu + phi * r_lag
        
        pm.Normal('likelihood', 
                 mu=mu_t, 
                 sigma=pm.math.sqrt(sigma_sq), 
                 observed=r_curr)
    
    return model


prior_spec = AR1_priors.get_pymc_spec()
ar1_bayes_results = {}
print("\nSampling Bayesian AR(1) models...\n")

for stock in stock_list:
    print(f"Sampling {stock}...")
    returns_train = train_df[stock].values
    model = build_ar1_bayesian_model(returns_train)
    
    with model:
        idata = pm.sample(
            draws=1000, 
            tune=500, 
            chains=2, 
            cores=4,           
            random_seed=42, 
            return_inferencedata=True,
            progressbar=False,
            target_accept=0.8  
        )
    
    ar1_bayes_results[stock] = {
        'model': model,
        'idata': idata,
        'returns_train': returns_train,
        'returns_test': test_df[stock].values
    }

# Extract results
ar1_bayes_summary_table = {}

for stock in stock_list:
    idata = ar1_bayes_results[stock]['idata']
    summary = az.summary(idata, var_names=['mu', 'phi', 'sigma_sq'])
    
    ar1_bayes_summary_table[stock] = {
        'mu_mean': summary.loc['mu', 'mean'],
        'mu_std': summary.loc['mu', 'sd'],
        'phi_mean': summary.loc['phi', 'mean'],
        'phi_std': summary.loc['phi', 'sd'],
        'sigma_sq_mean': summary.loc['sigma_sq', 'mean'],
        'sigma_sq_std': summary.loc['sigma_sq', 'sd'],
        'sigma_mean': np.sqrt(summary.loc['sigma_sq', 'mean']),
        'mu_hdi_low': summary.loc['mu', 'hdi_3%'],
        'mu_hdi_high': summary.loc['mu', 'hdi_97%'],
        'phi_hdi_low': summary.loc['phi', 'hdi_3%'],
        'phi_hdi_high': summary.loc['phi', 'hdi_97%'],
        'r_hat_mu': summary.loc['mu', 'r_hat'],
        'r_hat_phi': summary.loc['phi', 'r_hat'],
        'r_hat_sigma': summary.loc['sigma_sq', 'r_hat'],
        'ess_bulk_mu': summary.loc['mu', 'ess_bulk'],
        'ess_bulk_phi': summary.loc['phi', 'ess_bulk'],
        'ess_bulk_sigma': summary.loc['sigma_sq', 'ess_bulk'],
    }

ar1_bayes_df = pd.DataFrame(ar1_bayes_summary_table).T

print("\nAR(1) BAYESIAN POSTERIOR ESTIMATES:")
print(ar1_bayes_df[[
    'mu_mean', 'mu_std', 'phi_mean', 'phi_std', 'sigma_mean'
]].round(6).to_string())


print("\nCONVERGENCE DIAGNOSTICS - MCMC QUALITY METRICS:")
diagnostics_data = []
for stock in stock_list:
    summary = az.summary(ar1_bayes_results[stock]['idata'], 
                        var_names=['mu', 'phi', 'sigma_sq'])
    
    for param in ['mu', 'phi', 'sigma_sq']:
        diagnostics_data.append({
            'Stock': stock,
            'Parameter': param,
            'R_hat': summary.loc[param, 'r_hat'],
            'ESS_bulk': summary.loc[param, 'ess_bulk'],
            'ESS_tail': summary.loc[param, 'ess_tail'],
            'Mean': summary.loc[param, 'mean'],
            'Std': summary.loc[param, 'sd']
        })

diagnostics_df = pd.DataFrame(diagnostics_data)
print(diagnostics_df.to_string(index=False))

convergence_ok = (diagnostics_df['R_hat'] < 1.01).all()
if convergence_ok:
    print("\nAll parameters have converged (R_hat < 1.01).")
else:
    print("\nWARNING - Some parameters have not converged (R_hat >= 1.01).")

diagnostics_df.to_csv(f"{results_dirs['ar1_bayes']}/ar1_mcmc_diagnostics.csv", index=False)

print("\nEFFECTIVE SAMPLE SIZE (ESS) ANALYSIS:")
print("\nMinimum ESS by parameter:")
print(f"  mu:      {diagnostics_df[diagnostics_df['Parameter']=='mu']['ESS_bulk'].min():.0f}")
print(f"  phi:     {diagnostics_df[diagnostics_df['Parameter']=='phi']['ESS_bulk'].min():.0f}")
print(f"  sigma_sq: {diagnostics_df[diagnostics_df['Parameter']=='sigma_sq']['ESS_bulk'].min():.0f}")


print("\nGenerating MCMC trace plots...")
for stock in stock_list[:2]:
    idata = ar1_bayes_results[stock]['idata']
    
    fig, axes = plt.subplots(3, 2, figsize=(14, 10))
    
    for idx, param in enumerate(['mu', 'phi', 'sigma_sq']):
        # Trace
        ax = axes[idx, 0]
        chains = idata.posterior[param].values
        for chain in chains:
            ax.plot(chain, alpha=0.7, linewidth=0.8)
        ax.set_title(f'{stock}: {param} - Trace')
        ax.set_ylabel(param)
        ax.grid(True, alpha=0.3)
        
        # Posterior
        ax = axes[idx, 1]
        posterior = chains.flatten()
        ax.hist(posterior, bins=50, density=True, alpha=0.7, edgecolor='black')
        ax.axvline(posterior.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
        ax.set_title(f'{stock}: {param} - Posterior')
        ax.set_ylabel('Density')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{results_dirs['ar1_bayes']}/ar1_trace_posterior_{stock}.png", dpi=300, bbox_inches='tight')
    plt.close()

print("Trace plots saved.")

fig, axes = plt.subplots(2, 2, figsize=(14, 8))
for idx, stock in enumerate(stock_list[:2]):
    idata = ar1_bayes_results[stock]['idata']
    
    # Posterior mu
    ax = axes[idx, 0]
    posterior_mu = idata.posterior['mu'].values.flatten()
    ax.hist(posterior_mu, bins=40, density=True, alpha=0.7, color='steelblue', edgecolor='black')
    ax.axvline(posterior_mu.mean(), color='red', linestyle='--', linewidth=2)
    ax.set_title(f'{stock}: Posterior of mu')
    ax.set_ylabel('Density')
    ax.grid(True, alpha=0.3)
    
    # Posterior phi
    ax = axes[idx, 1]
    posterior_phi = idata.posterior['phi'].values.flatten()
    ax.hist(posterior_phi, bins=40, density=True, alpha=0.7, color='forestgreen', edgecolor='black')
    ax.axvline(posterior_phi.mean(), color='red', linestyle='--', linewidth=2)
    ax.set_title(f'{stock}: Posterior of phi')
    ax.set_ylabel('Density')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{results_dirs['ar1_bayes']}/ar1_posterior_distributions.png", dpi=300, bbox_inches='tight')
plt.close()

# Save results
ar1_bayes_df.to_csv(f"{results_dirs['ar1_bayes']}/ar1_bayesian_summary.csv")

print("AR(1) Bayesian estimation complete.")


Sampling Bayesian AR(1) models...

Sampling AAPL...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, phi, sigma_sq]
Sampling 2 chains for 500 tune and 1_000 draw iterations (1_000 + 2_000 draws total) took 24 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Sampling AMZN...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, phi, sigma_sq]
Sampling 2 chains for 500 tune and 1_000 draw iterations (1_000 + 2_000 draws total) took 16 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Sampling GOOGL...


...

Sampling NVDA...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, phi, sigma_sq]
Sampling 2 chains for 500 tune and 1_000 draw iterations (1_000 + 2_000 draws total) took 16 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


Sampling TSLA...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, phi, sigma_sq]
Sampling 2 chains for 500 tune and 1_000 draw iterations (1_000 + 2_000 draws total) took 16 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics



AR(1) BAYESIAN POSTERIOR ESTIMATES:
       mu_mean  mu_std  phi_mean  phi_std  sigma_mean
AAPL     0.001   0.001     0.002    0.032    0.000000
AMZN     0.000   0.001    -0.001    0.032    0.000000
GOOGL    0.001   0.001    -0.004    0.032    0.000000
META     0.001   0.001    -0.007    0.032    0.031623
MSFT     0.001   0.001    -0.016    0.032    0.000000
NVDA     0.002   0.001    -0.030    0.032    0.031623
TSLA     0.001   0.001    -0.028    0.033    0.031623

CONVERGENCE DIAGNOSTICS - MCMC QUALITY METRICS:
Stock Parameter  R_hat  ESS_bulk  ESS_tail   Mean   Std
 AAPL        mu    1.0    1437.0    1475.0  0.001 0.001
 AAPL       phi    1.0    2122.0    1609.0  0.002 0.032
 AAPL  sigma_sq    1.0    2354.0    1444.0  0.000 0.000
 AMZN        mu    1.0    1837.0    1595.0  0.000 0.001
 AMZN       phi    1.0    1746.0    1198.0 -0.001 0.032
 AMZN  sigma_sq    1.0    1922.0    1392.0  0.000 0.000
GOOGL        mu    1.0    2309.0    1656.0  0.001 0.001
GOOGL       phi    1.0    1953.0  

# 4: GARCH(1,1) - FREQUENTIST ESTIMATION

In [4]:
from arch import arch_model

garch_freq_results = {}
garch_freq_params = {}

print("\nFitting GARCH(1,1) models using Maximum Likelihood Estimation...\n")

for stock in stock_list:
    print(f"Fitting {stock}...")
    
    returns_train = train_df[stock].dropna().values
    
    model = arch_model(
        returns_train, 
        mean='Constant', 
        vol='Garch', 
        p=1, q=1,  
        rescale=False  
    )
    
    try:
        fit_res = model.fit(disp='off', show_warning=False)
        
        params = fit_res.params
        mu = params['mu']
        omega = params['omega']
        alpha = params['alpha[1]'] 
        beta = params['beta[1]']  
        
        long_run_var = omega / (1 - alpha - beta) if (alpha + beta) < 0.999 else np.nan
        
        garch_freq_results[stock] = {
            'fit': fit_res,
            'nobs': len(returns_train)
        }
        
        garch_freq_params[stock] = {
            'mu': mu,
            'omega': omega,
            'alpha': alpha,
            'beta': beta,
            'alpha_plus_beta': alpha + beta,
            'long_run_var': long_run_var,
            'aic': fit_res.aic,
            'bic': fit_res.bic,
            'log_likelihood': fit_res.loglikelihood
        }
        
        print(f"  mu={mu:.6f}, omega={omega:.6f}, alpha={alpha:.4f}, beta={beta:.4f}")
        
    except Exception as e:
        print(f"  Error fitting GARCH for {stock}: {e}")
        garch_freq_params[stock] = {'error': str(e)}

garch_freq_summary = pd.DataFrame(garch_freq_params).T
garch_freq_display = garch_freq_summary[['mu', 'omega', 'alpha', 'beta', 'alpha_plus_beta']]

print("\nGARCH(1,1) FREQUENTIST RESULTS:")
print(garch_freq_display.round(6).to_string())
garch_freq_summary.to_csv(f"{results_dirs['garch_bayes']}/garch_freq_params.csv")


Fitting GARCH(1,1) models using Maximum Likelihood Estimation...

Fitting AAPL...
  mu=0.000969, omega=0.000004, alpha=0.0343, beta=0.9525
Fitting AMZN...
  mu=0.000653, omega=0.000049, alpha=0.2000, beta=0.7000
Fitting GOOGL...
  mu=0.000890, omega=0.000007, alpha=0.0100, beta=0.9700
Fitting META...
  mu=0.001057, omega=0.000016, alpha=0.0100, beta=0.9700
Fitting MSFT...
  mu=0.000947, omega=0.000002, alpha=0.0275, beta=0.9656
Fitting NVDA...
  mu=0.002785, omega=0.000022, alpha=0.0100, beta=0.9700
Fitting TSLA...
  mu=0.000963, omega=0.000034, alpha=0.0338, beta=0.9434

GARCH(1,1) FREQUENTIST RESULTS:
             mu     omega     alpha      beta  alpha_plus_beta
AAPL   0.000969  0.000004  0.034318  0.952460         0.986778
AMZN   0.000653  0.000049  0.200000  0.700000         0.900000
GOOGL  0.000890  0.000007  0.010000  0.970000         0.980000
META   0.001057  0.000016  0.010024  0.969977         0.980001
MSFT   0.000947  0.000002  0.027469  0.965572         0.993041
NVDA   0.0

# 5: GARCH(1,1) - BAYESIAN ESTIMATION

In [ ]:
def build_garch_bayesian_model(returns):
    """
    GARCH(1,1) Bayesian Model with time-varying volatility
    r_t = mu + epsilon_t, epsilon_t ~ N(0, sqrt(h_t))
    h_t = omega + alpha * epsilon_{t-1}^2 + beta * h_{t-1}
    """
    prior_spec = GARCH_priors.get_pymc_spec()
    
    n_obs = len(returns)
    
    with pm.Model() as model:
        mu = pm.Normal('mu', **prior_spec['mu'][1])
        omega = pm.HalfNormal('omega', **prior_spec['omega'][1])
        alpha = pm.Beta('alpha', **prior_spec['alpha'][1])
        beta = pm.Beta('beta', **prior_spec['beta'][1])
        
        # Add the stationarity constraint (CRITICAL for GARCH)
        pm.Potential('stationarity_potential', 
                     pm.math.switch(alpha + beta < 1, 0, -np.inf))
        
        # Fixed h_init (pragmatic for speed)
        h_init = pt.as_tensor_variable(np.var(returns) * 0.1)
        
        residuals = returns - mu
        resid_sq = pt.square(residuals)

        # garch_step function for scan
        def garch_step(resid_t_sq, h_tm1, omega_, alpha_, beta_):
            h_t = omega_ + alpha_ * resid_t_sq + beta_ * h_tm1
            return h_t
        
        h_sequence, _ = scan(
            fn=garch_step,
            sequences=resid_sq[:-1],
            outputs_info=h_init,
            non_sequences=[omega, alpha, beta],
            n_steps=n_obs-1
        )
        
        h_full = pt.concatenate([[h_init], h_sequence])
        h_pos = pt.maximum(h_full, 1e-8) 
        
        pm.Normal('likelihood', mu=mu, sigma=pt.sqrt(h_pos), observed=returns)
    
    return model


garch_bayes_results = {}
garch_bayes_summary_table = {}


for stock in stock_list:
    print(f"\nSampling Bayesian GARCH(1,1) for {stock}...")
    
    returns_train = train_df[stock].dropna().values
    

    model = build_garch_bayesian_model(returns_train)
    
    with model:
        idata = pm.sample(
            draws=750,            
            tune=400,             
            chains=2,
            cores=2,
            random_seed=42,
            target_accept=0.80,  
            return_inferencedata=True,
            progressbar=True,
            nuts_sampler_kwargs={'max_treedepth': 10} 
        )
    
    summary = az.summary(idata, var_names=['mu', 'omega', 'alpha', 'beta'])
    
    alpha_plus_beta = summary.loc['alpha', 'mean'] + summary.loc['beta', 'mean']
    stationarity_ok = alpha_plus_beta < 0.999
    
    garch_bayes_results[stock] = {'idata': idata, 'model': model}
    
    garch_bayes_summary_table[stock] = {
        'mu_mean': summary.loc['mu', 'mean'],
        'mu_hdi': f"[{summary.loc['mu', 'hdi_3%']:.4f}, {summary.loc['mu', 'hdi_97%']:.4f}]",
        'omega_mean': summary.loc['omega', 'mean'],
        'alpha_mean': summary.loc['alpha', 'mean'],
        'beta_mean': summary.loc['beta', 'mean'],
        'alpha_plus_beta': alpha_plus_beta,
        'stationary': stationarity_ok,
        'r_hat_max': summary['r_hat'].max(),
        'ess_bulk_min': summary['ess_bulk'].min(),
        'mu_std': summary.loc['mu', 'sd'],
        'omega_std': summary.loc['omega', 'sd'],
        'alpha_std': summary.loc['alpha', 'sd'],
        'beta_std': summary.loc['beta', 'sd']
    }
    
    print(f"  Converged: R_hat={summary['r_hat'].max():.3f}, ESS={summary['ess_bulk'].min():.0f}")
    print(f"  Params: μ={summary.loc['mu', 'mean']:.6f}, "
            f"α={summary.loc['alpha', 'mean']:.3f}, β={summary.loc['beta', 'mean']:.3f}, "
            f"α+β={alpha_plus_beta:.3f} {'✓' if stationarity_ok else '✗'}")
              

garch_bayes_df = pd.DataFrame(garch_bayes_summary_table).T
print("BAYESIAN GARCH(1,1) SUMMARY (CORRECT TIME-VARYING VOLATILITY)")
display_cols = ['mu_mean', 'omega_mean', 'alpha_mean', 'beta_mean', 'alpha_plus_beta', 'stationary']
print(garch_bayes_df[display_cols].round(6))

garch_bayes_df.to_csv(f"{results_dirs['garch_bayes']}/garch_bayesian_correct.csv")


Sampling Bayesian GARCH(1,1) for AAPL...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, omega, alpha, beta]


Output()

Sampling 2 chains for 400 tune and 750 draw iterations (800 + 1_500 draws total) took 250 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


  Converged: R_hat=1.010, ESS=385
  Params: μ=0.001000, α=0.080, β=0.853, α+β=0.933 ✓

Sampling Bayesian GARCH(1,1) for AMZN...


...

  Converged: R_hat=1.010, ESS=419
  Params: μ=0.003000, α=0.051, β=0.904, α+β=0.955 ✓

Sampling Bayesian GARCH(1,1) for TSLA...


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, omega, alpha, beta]


Output()

Sampling 2 chains for 400 tune and 750 draw iterations (800 + 1_500 draws total) took 322 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


  Converged: R_hat=1.000, ESS=799
  Params: μ=0.001000, α=0.055, β=0.863, α+β=0.918 ✓
BAYESIAN GARCH(1,1) SUMMARY (CORRECT TIME-VARYING VOLATILITY)
      mu_mean omega_mean alpha_mean beta_mean alpha_plus_beta stationary
AAPL    0.001        0.0       0.08     0.853           0.933       True
AMZN    0.001        0.0       0.19     0.738           0.928       True
GOOGL   0.001        0.0      0.076     0.791           0.867       True
META    0.001        0.0      0.233     0.684           0.917       True
MSFT    0.001        0.0      0.047     0.933            0.98       True
NVDA    0.003        0.0      0.051     0.904           0.955       True
TSLA    0.001        0.0      0.055     0.863           0.918       True


# 6: FREQUENTIST vs BAYESIAN COMPARISON

In [10]:
# AR(1) Comparison
print("\nAR(1) MODEL COMPARISON:")

ar1_comparison = []
for stock in stock_list:
    ar1_comparison.append({
        'Stock': stock,
        'Freq_mu': ar1_freq_params[stock]['mu'],
        'Bayes_mu': ar1_bayes_df.loc[stock, 'mu_mean'],
        'Freq_phi': ar1_freq_params[stock]['phi'],
        'Bayes_phi': ar1_bayes_df.loc[stock, 'phi_mean'],
        'Freq_sigma': ar1_freq_params[stock]['sigma'],
        'Bayes_sigma': ar1_bayes_df.loc[stock, 'sigma_mean'],
    })

ar1_comp_df = pd.DataFrame(ar1_comparison).set_index('Stock')
print(ar1_comp_df.round(6).to_string())
ar1_comp_df.to_csv(f"{results_dirs['ar1_bayes']}/ar1_comparison.csv")

# GARCH(1,1) Comparison
print("\nGARCH(1,1) MODEL COMPARISON:")

garch_comparison = []
for stock in stock_list:
    if stock in garch_freq_params and stock in garch_bayes_df.index:
        if 'error' not in str(garch_bayes_df.loc[stock].values):
            garch_comparison.append({
                'Stock': stock,
                'Freq_mu': garch_freq_params[stock]['mu'],
                'Bayes_mu': garch_bayes_df.loc[stock, 'mu_mean'],
                'Freq_alpha': garch_freq_params[stock]['alpha'],
                'Bayes_alpha': garch_bayes_df.loc[stock, 'alpha_mean'],
                'Freq_beta': garch_freq_params[stock]['beta'],
                'Bayes_beta': garch_bayes_df.loc[stock, 'beta_mean'],
            })

garch_comp_df = pd.DataFrame(garch_comparison).set_index('Stock')
print(garch_comp_df.round(6).to_string())
garch_comp_df.to_csv(f"{results_dirs['garch_bayes']}/garch_comparison.csv")


AR(1) MODEL COMPARISON:
        Freq_mu  Bayes_mu  Freq_phi  Bayes_phi  Freq_sigma  Bayes_sigma
Stock                                                                  
AAPL   0.000700     0.001  0.001282      0.002    0.016902     0.000000
AMZN   0.000242     0.000  0.000464     -0.001    0.022156     0.000000
GOOGL  0.000696     0.001 -0.005942     -0.004    0.019166     0.000000
META   0.000726     0.001 -0.008070     -0.007    0.028492     0.031623
MSFT   0.000706     0.001 -0.016698     -0.016    0.016395     0.000000
NVDA   0.002476     0.002 -0.032073     -0.030    0.032823     0.031623
TSLA   0.000598     0.001 -0.029272     -0.028    0.037587     0.031623

GARCH(1,1) MODEL COMPARISON:
        Freq_mu  Bayes_mu  Freq_alpha  Bayes_alpha  Freq_beta  Bayes_beta
Stock                                                                    
AAPL   0.000969     0.001    0.034318        0.080   0.952460       0.853
AMZN   0.000653     0.001    0.200000        0.190   0.700000       0.738
G

# 7: SV Model

In [7]:
from pymc import math as pmm

def build_sv_bayesian_model_fast(returns, error_dist='normal'):
    """
    r_t = μ + exp(h_t/2) * ε_t
    h_t = α + φ * h_{t-1} + η_t
    
    REVISED: Priors adjusted to match typical financial data scale.
    """
    n_obs = len(returns)
    
    with pm.Model() as model:
        r = pm.Data("r", returns)
        T = r.shape[0]
        mu = pm.Normal('mu', mu=0., sigma=0.01)

        phi_raw = pm.Normal('phi_raw', mu=0., sigma=1.)
        phi = pm.Deterministic('phi', pmm.tanh(phi_raw))

    
        alpha = pm.Normal('alpha', mu=-8., sigma=1.0) 


        sigma_eta = pm.HalfNormal('sigma_eta', sigma=0.1) 

        z = pm.Normal('z', mu=0., sigma=1., shape=T)

        def ar1_step(z_t, h_prev, alpha_, phi_, sigma_eta_):
            h_t = alpha_ + phi_ * (h_prev - alpha_) + sigma_eta_ * z_t
            return h_t

        h_seq, _ = scan( 
            fn=ar1_step,
            sequences=[z],                       
            outputs_info=[alpha],               
            non_sequences=[alpha, phi, sigma_eta],
            n_steps=T
        )

        h_clip = pmm.clip(h_seq, -30, 30)

        # likelihood
        if error_dist == 'normal':
            pm.Normal('obs', mu=mu, sigma=pmm.exp(h_clip / 2), observed=r)
        elif error_dist == 'studentt':
            nu = pm.Exponential('nu', 1/30) 
            pm.StudentT('obs', nu=nu, mu=mu, sigma=pmm.exp(h_clip / 2), observed=r)
        else:
            raise ValueError("error_dist must be 'normal' or 'studentt'")

        pm.Deterministic('h_log_volatility_process', h_seq)

    return model

sv_results = {}
sv_summary_table = {}

print("SV with Non-Centered Parameterization")

for stock in stock_list:
    print(f"\n{stock}...", end=" ", flush=True)
    
    returns_train = train_df[stock].dropna().values
    n_obs = len(returns_train)
    print(f"(n={n_obs})", end=" | ")

    model = build_sv_bayesian_model_fast(returns_train)
    
    with model:
        idata = pm.sample(
            draws=500,         
            tune=500,          
            chains=2,           
            cores=2,            
            random_seed=42,
            target_accept=0.8, 
            max_treedepth=10,   
            return_inferencedata=True,
            progressbar=False,
            nuts_sampler_kwargs={'store_unconstrained_samples': False}
        )
    
    summary = az.summary(idata, var_names=['mu', 'alpha', 'phi', 'sigma_eta'])
    
    sv_results[stock] = {'idata': idata, 'model': model}
    
    mu_mean = summary.loc['mu', 'mean']
    alpha_mean = summary.loc['alpha', 'mean']
    phi_mean = summary.loc['phi', 'mean']
    sigma_eta_mean = summary.loc['sigma_eta', 'mean']
    vol_level = np.exp(alpha_mean / 2) * 100
    rhat_max = summary['r_hat'].max()
    ess_min = summary['ess_bulk'].min()
    div_count = idata.sample_stats.divergent.sum().item()
    
    sv_summary_table[stock] = {
        'mu': mu_mean,
        'alpha': alpha_mean,
        'phi': phi_mean,
        'sigma_eta': sigma_eta_mean,
        'vol_level_pct': vol_level,
        'rhat_max': rhat_max,
        'ess_bulk_min': ess_min,
        'divergences': div_count
    }
    
    status = "v" if (rhat_max < 1.05 and ess_min > 200 and div_count == 0) else "x"
    print(f"{status} φ={phi_mean:.3f}, σ_η={sigma_eta_mean:.3f}, Vol={vol_level:.1f}%")
    print(f"   R̂={rhat_max:.3f}, ESS={ess_min:.0f}, Div={div_count}")
        
print("SUMMARY")

sv_df = pd.DataFrame(sv_summary_table).T

if 'error' not in sv_df.columns:
    display_cols = ['phi', 'sigma_eta', 'vol_level_pct', 'rhat_max', 'ess_bulk_min', 'divergences']
    print(sv_df[display_cols].round(3).to_string())
else:
    print(sv_df.to_string())

sv_df.to_csv(f"{results_dirs['sv']}/sv_fast_results.csv")
print(f"\nResults saved to {results_dirs['sv']}/sv_fast_results.csv")


FAST SV with Non-Centered Parameterization (5-10 min per stock)

AAPL... (n=1003) | 

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, phi_raw, alpha, sigma_eta, z]


ValueError: Not enough samples to build a trace.

In [ ]:
from pymc import math as pmm
from pytensor.scan import scan
import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

def build_sv_bayesian_model_fast(returns, error_dist='normal'):
    """
    FAST & ACCURATE SV Model
    
    r_t = μ + exp(h_t/2) * ε_t
    h_t = α + φ * h_{t-1} + η_t
    
    Key improvements:
    - Adjusted priors for financial data
    - Non-centered parameterization
    - Proper initialization via MAP
    - Numerical stability
    """
    n_obs = len(returns)
    
    with pm.Model() as model:
        r = pm.Data("r", returns)
        T = r.shape[0]
        
        # ✅ OBSERVATION MEAN: tight prior (financial returns ~0)
        mu = pm.Normal('mu', mu=0., sigma=0.01)

        # ✅ PERSISTENCE (phi): Beta reparameterization for (0, 1) support
        # phi ~ Beta(20, 1.5) gives mode ~0.93, concentrated on (0.8, 1.0)
        phi_raw = pm.Beta('phi_raw', alpha=20, beta=1.5)
        phi = pm.Deterministic('phi', 0.9 + 0.099 * phi_raw)  # Rescale to [0.9, 0.999]

        # ✅ INTERCEPT: centered at log(variance) ~ -8 to -4
        alpha = pm.Normal('alpha', mu=-6.0, sigma=1.5)

        # ✅ VOLATILITY OF VOLATILITY: HalfNormal is good
        sigma_eta = pm.HalfNormal('sigma_eta', sigma=0.2)

        # ✅ NON-CENTERED LATENT INNOVATIONS
        z = pm.Normal('z', mu=0., sigma=1., shape=T)

        # ✅ AR(1) RECURSION FUNCTION
        def ar1_step(z_t, h_prev, alpha_, phi_, sigma_eta_):
            """
            Centered parameterization:
            h_t = α + φ*(h_{t-1} - α) + σ_η*z_t
            """
            h_t = alpha_ + phi_ * (h_prev - alpha_) + sigma_eta_ * z_t
            return h_t

        # ✅ INITIAL STATE: stationary distribution mean
        h0 = alpha

        # ✅ SCAN: Build latent volatility sequence
        h_seq, _ = scan(
            fn=ar1_step,
            sequences=[z],
            outputs_info=[h0],
            non_sequences=[alpha, phi, sigma_eta],
            n_steps=T
        )

        # ✅ NUMERICAL STABILITY: clip to [-30, 20]
        h_clip = pmm.clip(h_seq, -30, 20)

        # ✅ OBSERVATION LIKELIHOOD
        if error_dist == 'normal':
            pm.Normal('obs', mu=mu, sigma=pmm.exp(h_clip / 2), observed=r)
        elif error_dist == 'studentt':
            nu = pm.Exponential('nu', 1/30)
            pm.StudentT('obs', nu=nu, mu=mu, sigma=pmm.exp(h_clip / 2), observed=r)
        else:
            raise ValueError("error_dist must be 'normal' or 'studentt'")

        # ✅ EXPOSE LATENT VOLATILITY FOR DIAGNOSTICS
        pm.Deterministic('h_seq', h_seq)

    return model


def fit_sv_with_map_init(returns, stock_name='', verbose=True):
    """
    Fit SV model with MAP initialization for speed & stability
    """
    n_obs = len(returns)
    
    if verbose:
        print(f"  Building model (n={n_obs})...", end=" ", flush=True)
    
    model = build_sv_bayesian_model_fast(returns)
    
    with model:
        # ✅ STEP 1: Fast MAP estimation (30-60 sec)
        if verbose:
            print("MAP...", end=" ", flush=True)
        
        try:
            map_est = pm.find_MAP(maxeval=3000, include_transformed=True)
        except Exception as e:
            if verbose:
                print(f"MAP failed ({str(e)[:30]}), using random init...", end=" ", flush=True)
            map_est = None
        
        # ✅ STEP 2: NUTS Sampling (2-5 min)
        if verbose:
            print("NUTS...", end=" ", flush=True)
        
        idata = pm.sample(
            draws=800,          # More draws for better ESS
            tune=600,           # More tuning
            chains=2,
            cores=2,
            random_seed=42,
            target_accept=0.85,  # Higher for complex models
            max_treedepth=12,    # Allow deeper trees
            return_inferencedata=True,
            progressbar=False,
            nuts_sampler_kwargs={'store_unconstrained_samples': False},
            initvals=map_est if map_est else None,  # Use MAP if available
            discard_tuned_samples=True
        )
        
        if verbose:
            print("done!", flush=True)
    
    return model, idata


# ============================================================================
# MAIN FITTING LOOP
# ============================================================================

sv_results = {}
sv_summary_table = {}
sv_diagnostics = {}

print("\n" + "="*100)
print("FAST & ACCURATE STOCHASTIC VOLATILITY MODEL")
print("="*100 + "\n")

# Track timing
import time
start_total = time.time()

for idx, stock in enumerate(stock_list, 1):
    print(f"[{idx}/{len(stock_list)}] {stock}...", end=" ", flush=True)
    
    returns_train = train_df[stock].dropna().values
    n_obs = len(returns_train)
    
    start_stock = time.time()
    
    try:
        # ✅ FIT MODEL
        model, idata = fit_sv_with_map_init(returns_train, stock, verbose=False)
        
        # ✅ EXTRACT SUMMARY STATISTICS
        summary = az.summary(idata, var_names=['mu', 'alpha', 'phi', 'sigma_eta'])
        
        mu_mean = summary.loc['mu', 'mean']
        alpha_mean = summary.loc['alpha', 'mean']
        phi_mean = summary.loc['phi', 'mean']
        sigma_eta_mean = summary.loc['sigma_eta', 'mean']
        
        # ✅ COMPUTE DERIVED QUANTITIES
        vol_level = np.exp(alpha_mean / 2) * 100  # Long-run vol in %
        
        # ✅ DIAGNOSTICS
        rhat_max = summary['r_hat'].max()
        ess_bulk_min = summary['ess_bulk'].min()
        ess_tail_min = summary['ess_tail'].min()
        div_count = idata.sample_stats.divergent.sum().item()
        
        # ✅ MCMC DIAGNOSTICS
        n_eff_ratio = ess_bulk_min / (2 * 800)  # 2 chains, 800 draws
        
        # ✅ CONVERGENCE CHECK
        converged = (rhat_max < 1.01) and (ess_bulk_min > 400) and (div_count == 0)
        status = "✓" if converged else "⚠"
        
        # ✅ STORE RESULTS
        sv_results[stock] = {
            'model': model,
            'idata': idata,
            'n_obs': n_obs
        }
        
        sv_summary_table[stock] = {
            'mu': mu_mean,
            'alpha': alpha_mean,
            'phi': phi_mean,
            'sigma_eta': sigma_eta_mean,
            'vol_pct': vol_level,
            'rhat_max': rhat_max,
            'ess_bulk_min': ess_bulk_min,
            'ess_tail_min': ess_tail_min,
            'divergences': int(div_count),
            'eff_sample_ratio': n_eff_ratio
        }
        
        sv_diagnostics[stock] = {
            'converged': converged,
            'rhat_ok': rhat_max < 1.01,
            'ess_ok': ess_bulk_min > 400,
            'div_ok': div_count == 0,
            'eff_ok': n_eff_ratio > 0.5
        }
        
        elapsed = time.time() - start_stock
        
        # ✅ PRINT RESULTS
        print(f"{status} φ={phi_mean:.4f}, σ_η={sigma_eta_mean:.4f}, Vol={vol_level:.2f}%", end="")
        print(f" | R̂={rhat_max:.4f}, ESS={ess_bulk_min:.0f}, Div={int(div_count)} | {elapsed:.1f}s")
        
    except Exception as e:
        print(f"✗ ERROR: {str(e)[:80]}")
        sv_summary_table[stock] = {'error': str(e)[:100]}
        sv_diagnostics[stock] = {'converged': False, 'error': True}
        import traceback
        traceback.print_exc()

elapsed_total = time.time() - start_total

# ============================================================================
# SUMMARY TABLE
# ============================================================================

print("\n" + "="*100)
print("RESULTS SUMMARY")
print("="*100 + "\n")

sv_df = pd.DataFrame(sv_summary_table).T

# Display key columns
display_cols = ['phi', 'sigma_eta', 'vol_pct', 'rhat_max', 'ess_bulk_min', 'divergences', 'eff_sample_ratio']
valid_cols = [c for c in display_cols if c in sv_df.columns]

print(sv_df[valid_cols].round(4).to_string())

# ============================================================================
# DIAGNOSTICS SUMMARY
# ============================================================================

print("\n" + "="*100)
print("CONVERGENCE DIAGNOSTICS")
print("="*100 + "\n")

diag_df = pd.DataFrame(sv_diagnostics).T
print(diag_df.to_string())

n_converged = diag_df['converged'].sum()
print(f"\n✓ {n_converged}/{len(stock_list)} models converged successfully")
print(f"✓ Total time: {elapsed_total/60:.1f} minutes ({elapsed_total/len(stock_list):.1f} min/stock)")

# ============================================================================
# SAVE RESULTS
# ============================================================================

sv_df.to_csv(f"{results_dirs['sv']}/sv_fast_results.csv")
diag_df.to_csv(f"{results_dirs['sv']}/sv_diagnostics.csv")

print(f"\n✓ Results saved:")
print(f"  - {results_dirs['sv']}/sv_fast_results.csv")
print(f"  - {results_dirs['sv']}/sv_diagnostics.csv")

# ============================================================================
# EXTRACT & STORE LATENT VOLATILITIES
# ============================================================================

print("\n" + "="*100)
print("EXTRACTING LATENT VOLATILITIES")
print("="*100 + "\n")

sv_latent_vols = {}

for stock in sv_results.keys():
    try:
        idata = sv_results[stock]['idata']
        h_seq_posterior = idata.posterior['h_seq'].values  # Shape: (chains, draws, time)
        h_mean = h_seq_posterior.mean(axis=(0, 1))  # Average over chains and draws
        h_std = h_seq_posterior.std(axis=(0, 1))
        
        vol_mean = np.exp(h_mean / 2) * 100  # Convert to %
        vol_lower = np.exp((h_mean - 1.96*h_std) / 2) * 100
        vol_upper = np.exp((h_mean + 1.96*h_std) / 2) * 100
        
        sv_latent_vols[stock] = {
            'vol_mean': vol_mean,
            'vol_lower': vol_lower,
            'vol_upper': vol_upper,
            'h_seq': h_mean
        }
        
        print(f"{stock}: mean vol={vol_mean.mean():.2f}%, range=[{vol_lower.mean():.2f}%, {vol_upper.mean():.2f}%]")
        
    except Exception as e:
        print(f"{stock}: ✗ Error extracting volatility ({str(e)[:50]})")

# ============================================================================
# QUALITY CHECKS
# ============================================================================

print("\n" + "="*100)
print("QUALITY ASSURANCE")
print("="*100 + "\n")

quality_checks = {
    'R̂ < 1.01': (sv_df['rhat_max'] < 1.01).sum(),
    'ESS_bulk > 400': (sv_df['ess_bulk_min'] > 400).sum(),
    'No divergences': (sv_df['divergences'] == 0).sum(),
    'Effective sample ratio > 0.5': (sv_df['eff_sample_ratio'] > 0.5).sum(),
}

for check, count in quality_checks.items():
    pct = 100 * count / len(sv_df)
    status = "✓" if pct >= 80 else "⚠"
    print(f"{status} {check}: {count}/{len(sv_df)} ({pct:.0f}%)")

print("\n✓ SV model fitting complete!")



FAST & ACCURATE STOCHASTIC VOLATILITY MODEL

[1/7] AAPL... 

Output()

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, phi_raw, alpha, sigma_eta, z]


✗ ERROR: Not enough samples to build a trace.
[2/7] AMZN... 

Traceback (most recent call last):
  File "C:\Users\natha\AppData\Local\Temp\ipykernel_3240\1922139345.py", line 160, in <module>
    model, idata = fit_sv_with_map_init(returns_train, stock, verbose=False)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\natha\AppData\Local\Temp\ipykernel_3240\1922139345.py", line 113, in fit_sv_with_map_init
    idata = pm.sample(
            ^^^^^^^^^^
  File "c:\Users\natha\anaconda3\envs\bayesian_final\Lib\site-packages\pymc\sampling\mcmc.py", line 957, in sample
    return _sample_return(
           ^^^^^^^^^^^^^^^
  File "c:\Users\natha\anaconda3\envs\bayesian_final\Lib\site-packages\pymc\sampling\mcmc.py", line 1042, in _sample_return
    traces, length = _choose_chains(traces, tune)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\natha\anaconda3\envs\bayesian_final\Lib\site-packages\pymc\backends\base.py", line 624, in _choose_chains
    raise ValueError("Not enough samples to bui

Output()

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, phi_raw, alpha, sigma_eta, z]
